In [1]:
!pip install dgl
!pip install jenkspy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import random
from jenkspy import JenksNaturalBreaks

import os
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import dgl
from dgl.nn import SAGEConv
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sn
import warnings
import scipy.stats as stats

warnings.filterwarnings('ignore')
random.seed(1)

In [3]:
class MyLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(MyLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)


    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        # 初始化hidden和memory cell参数
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)

        # forward propagate lstm
        out, (h_n, h_c) = self.lstm(x, (h0, c0))

        # 选取最后一个时刻的输出
        out = out[:, -1, :]
        return out

In [4]:
class MyDataset(Dataset):
    '''
    自定义数据集
    '''
    def __init__(self, features, labels, graph):
        self.features = features
        self.labels = labels
        self.graph = graph
        # self.altitudes = altitudes

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        graph = self.graph[idx]
        label = self.labels[idx]
        # altitude = self.altitudes[idx]
        return feature, graph, label


def get_each_person(data):
    '''
    将所有数据按照85/人划分
    :param data:
    :return:
    '''
    i_count = 0
    all_person, one_person = [], []
    for index, values in data.iterrows():
        i_count += 1
        if i_count % 85 == 0:
            one_person.append(values)
            all_person.append(one_person)
            one_person = []
        else:
            one_person.append(values)
    return all_person


def get_label_class(values):
    '''
    归一化后的标签离散处理
    :param values: 归一化后的标签值
    :return:
    '''

    # return int(values[0]*9)

    bins = [0.5, 0.6, 1.1]  # 暂时划分为三个等级
    for i in range(len(bins)):
        if bins[i] > values[0]:
            return i


def get_graph(graph):
    '''
    构图
    :param graph: 矩阵14*14
    :return:
    '''
    index = [[], []]
    i = 0
    for line in graph:
        j = 0
        for node in line:
            if i == j:
                break
            if node == 1:
                index[0].append(i)
                index[1].append(j)
            j+=1
        i += 1
    g = dgl.graph((index[0], index[1]))
    g = dgl.add_self_loop(g)
    return g


def get_altitude_gap(data_edm):
    altitude = list(data_edm['dem'].values)
    gaps = [altitude[-1]-altitude[0]]
    for i in range(len(altitude)-1):
        gaps.append(altitude[i+1]-altitude[i])
    zscores = stats.zscore(gaps)
    return zscores.astype(np.float32)


def read_data(path_fea, path_label, path_graph):
    '''
    加载原始数据
    :param path_fea:
    :param path_label:
    :param path_graph:
    :return:
    '''
    features, labels, graphs, altitudes = [], [], [], []
    data_fea = pd.read_csv(path_fea)
    data_label = pd.read_csv(path_label, header=None).T
    data_graph = pd.DataFrame(np.load(path_graph).reshape((len(data_label),-1)))
    # data_edm = pd.read_csv(path_dem, index_col=0)

    per_fea = get_each_person(data_fea)
    per_label = get_each_person(data_label)
    per_graph = get_each_person(data_graph)
    # altitude_gap = get_altitude_gap(data_edm)

    time_window_node, time_window_lstm = 5, 5
    for p in range(len(per_fea)):
        insert_0 = per_fea[p][0]
        for _ in range(time_window_node):
            per_fea[p].insert(0, insert_0)

        data_label = pd.DataFrame(per_label[p])
        data_label_norm = (data_label - data_label.min()) / (data_label.max() - data_label.min())

        # data_label_norm = pd.qcut(x=data_label_norm[0], q=3, labels=range(0,3), duplicates="drop")
        jnb =JenksNaturalBreaks(3)
        jnb.fit(list(data_label_norm[0].values))
        data_label_norm = jnb.labels_
        data_fea = pd.DataFrame(per_fea[p])
        data_graph = pd.DataFrame(per_graph[p])

        lstm_fea, graph_ = [], []
        for i in range(len(data_fea)-time_window_node):
            fea = data_fea.iloc[i:i+time_window_node].values
            lstm_fea.append(fea.astype(np.float32))

        for i in range(len(lstm_fea)-time_window_lstm):
            features.append(np.array(lstm_fea[i:i+time_window_lstm]).astype(np.float32))
            graphs.append(data_graph.iloc[i:i+time_window_lstm].values.reshape(time_window_lstm, 14,-1))
            # altitudes.append(altitude_gap[i:i + time_window_lstm])
            # class_ = get_label_class(data_label_norm.iloc[i+time_window_lstm])
            class_ = data_label_norm[i+time_window_lstm]
            labels.append(class_)

    # features, labels, graphs, altitudes = shuffle(features, labels, graphs, altitudes)
    features, labels, graphs = shuffle(features, labels, graphs)

    return features, labels, graphs


def shuffle(features, labels, graphs):
    '''
    打乱顺序
    :param features:
    :param labels:
    :param graphs:
    :return:
    '''
    new_features, new_labels, new_graphs = [], [], []
    index = [i for i in range(len(features))]
    random.shuffle(index)
    for i in index:
        new_features.append(features[i])
        new_labels.append(labels[i])
        new_graphs.append(graphs[i])
        # new_altitudes.append(altitudes[i])

    return new_features, new_labels, new_graphs



class My_model(nn.Module):
    def __init__(self):
        super(My_model, self).__init__()
        self.graph_embedding = SAGEConv(5, 128, 'pool')
        self.flat = nn.Linear(128*14, 128)
        self.lstm = MyLSTM(input_size=128, hidden_size=128, num_layers=1)
        self.classify = nn.Linear(128, 3)
        # self.altitude_emd = nn.Linear(8, 128)

        # self.emb = nn.Linear(12*5, 128)

        # self.ac = nn.Sigmoid()
        self.softmax = nn.Softmax()

    def forward(self, features, graph):
        features = features.permute(0,1,3,2)
        batch, time_step, node, channel = features.shape

        # cov_fea_tensor = self.emb(features.reshape(batch, time_step, -1))

        cov_fea = []
        for i in range(batch):
            lstm = []
            for j in range(time_step):
                fea = features[i, j, :, :]
                g = get_graph(graph[i][j])
                graph_cov = self.graph_embedding(g, fea).view(-1)
                fea_flat = self.flat(graph_cov)
                lstm.append(fea_flat)
            lstm = torch.stack(lstm, dim=0)
            cov_fea.append(lstm)
        cov_fea_tensor = torch.stack(cov_fea, dim=0)

        lstm_fea = self.lstm(cov_fea_tensor)
        # lstm_fea = self.ac(lstm_fea)

        # altitude_emd = self.altitude_emd(altitude)
        # concat = torch.cat([altitude_emd,lstm_fea], dim=1)
        y_class = self.classify(lstm_fea)
        y_class_softmax = self.softmax(y_class)
        return y_class



def val_model(model, val_dataloader):
    y_truths, y_preds, val_loss = [], [], []
    with torch.no_grad():
        model.eval()
        for batch in val_dataloader:
            y = batch[2]
            y_pre = model.forward(batch[0], batch[1])

            loss = loss_fun(y_pre, y)
            val_loss.append(loss.item())
            pred_i = y_pre.data.max(1, keepdim=True)[1]
            y_truths.append(y.numpy()[0])
            y_preds.append(pred_i.numpy()[0][0])
    val_loss = np.mean(val_loss)
    acc = metrics.accuracy_score(y_truths, y_preds)
    return acc, val_loss


def train_model(model, loss_fun, optimizer, train_dataloader, val_dataloader, epochs):
    best_epoch = 1
    for epo in range(epochs):
        losses = []
        for batch in train_dataloader:
            y = batch[2]
            y_dot = model.forward(batch[0], batch[1])
            loss = loss_fun(y_dot, y)
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
        train_loss = np.mean(losses)
        val_acc, val_loss = val_model(model, val_dataloader)

        if val_acc < best_epoch:
            best_epoch = val_acc
            torch.save(model.state_dict(), 'epoch_best.pkl')

        print('Epochs: %s/%s  Train loss: %.6f  Val loss: %.6f verification accuracy： %.6f' % (epo, epochs, train_loss, val_loss, val_acc))

    print("Train finished !")
    return model


def test_model(model, test_dataloader):
    y_truths, y_preds = [], []
    with torch.no_grad():
        model.eval()
        for batch in test_dataloader:
            y = batch[2]
            y_pre = model.forward(batch[0], batch[1])

            pred_i = y_pre.data.max(1, keepdim=True)[1]
            y_truths.append(y.numpy()[0])
            y_preds.append(pred_i.numpy()[0][0])
    acc = metrics.accuracy_score(y_truths, y_preds)
    confusion_matrix = metrics.confusion_matrix(y_truths, y_preds)
    print(acc)
    print(confusion_matrix)
    return confusion_matrix



In [8]:
if __name__ == '__main__':
    path_fea = 'zscore_new_23-08-v2.csv'
    path_label = 'label-v2.csv'
    path_graph = 'spatial_interactive_graphs_v3.npy'
    # path_dem = 'dem.csv'

    features, labels, graphs = read_data(path_fea, path_label, path_graph)
    # for i in features:
    #   print(i.shape)
    # df_labels=pd.DataFrame(labels)
    # df_labels.plot.density()
    features=stats.zscore(features)

    epochs = 100
    batch_size = 128
    lr = 0.0001

    train_per = 0.7
    val_per = 0.1
    train_num = int(len(features)*train_per)
    val_num = int(len(features)*val_per)

    dataset_train = MyDataset(features[:train_num], labels[:train_num], graphs[:train_num])
    # dataset_val = MyDataset(features[train_num:train_num+val_num], labels[train_num:train_num+val_num], graphs[train_num:train_num+val_num])
    dataset_test = MyDataset(features[train_num:], labels[train_num:], graphs[train_num:])

    train_dataloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(dataset_test)
    test_dataloader = DataLoader(dataset_test)

    model = My_model()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)  # config.learning_rate
    loss_fun = nn.CrossEntropyLoss()

    train = True
    if train:
        train_model(model, loss_fun, optimizer, train_dataloader, val_dataloader, epochs)
    else:
        model.load_state_dict(torch.load('epoch_best.pkl'))

    confusion_matrix = test_model(model, test_dataloader)

    # sn.heatmap(confusion_matrix, annot=True)
    # plt.xlabel('Predict')
    # plt.ylabel('Truth')
    # plt.show()

Epochs: 0/100  Train loss: 1.048282  Val loss: 1.059211 verification accuracy： 0.438889
Epochs: 1/100  Train loss: 1.043524  Val loss: 1.047480 verification accuracy： 0.490278
Epochs: 2/100  Train loss: 1.034283  Val loss: 1.007124 verification accuracy： 0.481944
Epochs: 3/100  Train loss: 1.045569  Val loss: 1.058080 verification accuracy： 0.483333
Epochs: 4/100  Train loss: 1.062031  Val loss: 1.030557 verification accuracy： 0.476389
Epochs: 5/100  Train loss: 1.003274  Val loss: 0.988275 verification accuracy： 0.484722
Epochs: 6/100  Train loss: 1.004822  Val loss: 1.005825 verification accuracy： 0.470833
Epochs: 7/100  Train loss: 0.998543  Val loss: 0.991265 verification accuracy： 0.500000
Epochs: 8/100  Train loss: 0.985316  Val loss: 0.984146 verification accuracy： 0.497222
Epochs: 9/100  Train loss: 0.962275  Val loss: 0.949244 verification accuracy： 0.536111
Epochs: 10/100  Train loss: 0.971470  Val loss: 0.953669 verification accuracy： 0.508333
Epochs: 11/100  Train loss: 0.9